In [1]:
import os
import numpy as np
import cv2 as cv
# from utils import Utils

In [2]:

menPath = "../Dataset_0-5/men/"
womenPath = "../Dataset_0-5/Women/"

In [3]:

def getThresholdedHand(frame):
    # Convert image to HSV
    hsvim = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    # Lower boundary of skin color in HSV
    lower = np.array([0, 48, 80], dtype="uint8")
    # Upper boundary of skin color in HSV
    upper = np.array([20, 255, 255], dtype="uint8")
    skinMask = cv.inRange(hsvim, lower, upper)

    # Gaussian filter (blur) to remove noise
    skinMask = cv.GaussianBlur(skinMask, (17, 17), 0)

    # get thresholded image
    # ret, thresh1 = cv.threshold(
    # skinMask, 100, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)
    thresh1 = cv.adaptiveThreshold(
        skinMask, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 355, 5)

    return thresh1

In [4]:

# # while True:
# class_dir = os.path.join(menPath, f"{0}")
# imgPath = os.path.join(class_dir, f'{0}_men ({1}).JPG')
# img = cv.imread(imgPath)
# img = getThresholdedHand(img)
# cv.imshow("img", img)
# cv.waitKey(0)
# cv.destroyAllWindows()

In [28]:
def getMaskedHand2(frame):
        # Convert image to Lab
        labImg = cv.cvtColor(frame, cv.COLOR_RGB2LAB)
        # Lower boundary of skin color in Lab
        lower = np.array([35, -20, -25], dtype="uint8")
        # Upper boundary of skin color in Lab
        upper = np.array([95, 25, 30], dtype="uint8")
        # Thresholding the image
        # skinMask = cv.inRange(labImg, lower, upper)
        # # Gaussian filter (blur) to remove noise
        # skinMask = cv.GaussianBlur(skinMask, (17, 17), 0)
        
        # greyImg = cv.cvtColor(cv.cvtColor(skinMask, cv.COLOR_Lab2RGB),cv.COLOR_RGB2GRAY)
        # greyImg = cv.normalize(greyImg, None, alpha=0, beta=255, norm_type=cv.NORM_MINMAX)
        # masked = cv.bitwise_and(greyImg, greyImg, mask=skinMask)
        # maskedBlurred = cv.GaussianBlur(masked, (41, 41), 0)
        return labImg

In [30]:
out = getMaskedHand2(cv.imread("D:/CMP/third_Year/second_semester/pattern Recognition/project/Dataset_0-5/men/0/0_men (2).JPG"))
cv.imshow("out", cv.resize(out, (1024, 512)))
cv.waitKey(0)
cv.destroyAllWindows()